In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

URL = 'https://www.gurufocus.com/stock/'
HEADERS = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
           'accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}


def get_html(url, params=None):
  return requests.get(url, headers=HEADERS, params=params)


def get_content(html):
  soup = BeautifulSoup(html, 'html.parser')
  data = soup.find_all('tr', class_='stock-indicators-table-row')
  ind = -1
  for i in range(len(data)):
    row_name = data[i].find_all('a')[0].contents[0].strip()
    if row_name == 'Piotroski F-Score':
      ind = i
  if ind == -1:
    return None
  else:
    pfs = data[ind].contents[2].contents[0]
    return pfs


def parse(tic):
  html = get_html(URL + tic + '/summary')
  if html.status_code == 200:
    pfs = get_content(html.text)
    return tic, pfs
  else:
    print('Error')
    return tic, None


df = pd.read_csv('/spb_stocks.csv')
tickers = [el.lower() for el in df.tiker]



data = []
for ticker in tickers:
  pfs = parse(ticker)
  data.append(pfs)

print(data)

In [0]:
df = pd.DataFrame(data)
df.rename(mapper={0: 'ticker', 1: 'pfs'}, inplace=True, axis=1)
df.to_csv('/pfs_table.csv', index=False)